In [1]:
import geopandas as gpd
import pandas as pd
import requests
from shapely.geometry import Point
import numpy as np
from geopy.distance import geodesic

In [4]:
df = pd.read_csv("../../data/processed/scoring_indicators/housing_needs_characteristics/merged_housing_data.csv")

In [6]:
df

,2020 Census Tract,geoid_,County_Name,pct_severe_housing_problems,avg_pop_yoy_growth_2018_2021,pop_2021_gt_2011,avg_emp_growth_2020_2022,qct?
0,13001950100,05000US13001,"Appling County, Georgia",0.204082,0.000995,Yes,0.0125,No
1,13001950201,05000US13003,"Atkinson County, Georgia",0.000000,0.000180,No,-0.0050,No
2,13001950202,05000US13005,"Bacon County, Georgia",0.177419,-0.001929,Yes,0.0105,Yes
3,13001950301,05000US13007,"Baker County, Georgia",0.309524,-0.027904,No,0.0050,No
4,13001950302,05000US13009,"Baldwin County, Georgia",0.236842,-0.010421,No,0.0165,No
...,...,...,...,...,...,...,...,...
2791,13321950201,NaN,NaN,NaN,NaN,NaN,NaN,No
2792,13321950202,NaN,NaN,0.574468,NaN,NaN,NaN,No
2793,13321950400,NaN,NaN,0.181395,NaN,NaN,NaN,No
2794,13321950500,NaN,NaN,0.100000,NaN,NaN,NaN,No


In [5]:
census_tracts = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp")
ga_tracts = census_tracts.to_crs(epsg=4326)


In [12]:
def find_census_tract(lat, lon, tracts_gdf):
    point = Point(lon, lat)
    match = tracts_gdf[tracts_gdf.contains(point)]
    if not match.empty:
        return str(match.iloc[0]["GEOID"])
    else:
        return None

In [10]:
def compute_housing_need_score(df, tract_geoid):
    row = df[df['2020 Census Tract'].astype(str) == tract_geoid]
    if row.empty:
        return 0  # no data for tract

    row = row.iloc[0]

    # Conditions from QAP
    not_qct = str(row["qct?"]).strip().lower() == "no"
    severe_housing = row["pct_severe_housing_problems"] >= 0.45
    pop_grew = str(row["pop_2021_gt_2011"]).strip().lower() == "yes"
    pop_growth = row["avg_pop_yoy_growth_2018_2021"] > 0.01
    emp_growth = row["avg_emp_growth_2020_2022"] > 0.01

    county_qualifies = (pop_grew and pop_growth) or emp_growth

    if not_qct and severe_housing and county_qualifies:
        return 5
    else:
        return 0

In [ ]:
# Site Latitude and Longitude
latitude = 33.902836
longitude = -84.205688

actual_tract = find_census_tract(latitude, longitude, ga_tracts)
print(tract_info)
score_info = compute_housing_need_score(df, actual_tract)
print(score_info)

5
